In [1]:
import os
import clang
from clang.cindex import *
from copy import deepcopy

In [2]:
# clang.cindex.Config.set_library_file('/usr/lib/x86_64-linux-gnu/libclang-14.so.14.0.0')
# Config.set_library_file("/usr/lib/x86_64-linux-gnu/libLLVM-14.so.1")
# Config.set_library_file("/usr/lib/x86_64-linux-gnu/libstdc++.so.6")
Config.set_library_file("C:\\Users\\Hp\\anaconda3\\envs\\tf\\Lib\\site-packages\\clang\\native\\libclang.dll")

In [3]:
# create an index and parse the source code
index = Index.create()
root_cursor = index.parse("main.c").cursor
# root_cursor = index.parse("/home/dipu/Documents/AI/MinorProject/c-corpus/cleaned/gno/lib/libc/tests/sys/rename.c").cursor

In [4]:
def print_ast(node, indent):
    try:
        print(" "*indent + node.spelling + " " + str(node.kind) + " " + str(node.type.spelling))
        for c in node.get_children():
            print_ast(c, indent+2)
    except ValueError:
        pass

print_ast(root_cursor, 0)

main.c CursorKind.TRANSLATION_UNIT 
  uintptr_t CursorKind.TYPEDEF_DECL uintptr_t
  va_list CursorKind.TYPEDEF_DECL va_list
  __va_start CursorKind.FUNCTION_DECL void (char **, ...)
     CursorKind.PARM_DECL va_list *
      va_list CursorKind.TYPE_REF va_list
  size_t CursorKind.TYPEDEF_DECL size_t
  ptrdiff_t CursorKind.TYPEDEF_DECL ptrdiff_t
  intptr_t CursorKind.TYPEDEF_DECL intptr_t
  __vcrt_bool CursorKind.TYPEDEF_DECL __vcrt_bool
  wchar_t CursorKind.TYPEDEF_DECL wchar_t
  __security_init_cookie CursorKind.FUNCTION_DECL void (void)
  __security_check_cookie CursorKind.FUNCTION_DECL void (uintptr_t)
    _StackCookie CursorKind.PARM_DECL uintptr_t
      uintptr_t CursorKind.TYPE_REF uintptr_t
  __report_gsfailure CursorKind.FUNCTION_DECL void (uintptr_t) __attribute__((noreturn))
    _StackCookie CursorKind.PARM_DECL uintptr_t
      uintptr_t CursorKind.TYPE_REF uintptr_t
  __security_cookie CursorKind.VAR_DECL uintptr_t
    uintptr_t CursorKind.TYPE_REF uintptr_t
  __crt_bool Curs

           CursorKind.PAREN_EXPR va_list
             CursorKind.BINARY_OPERATOR va_list
              _ArgList CursorKind.DECL_REF_EXPR va_list
               CursorKind.CSTYLE_CAST_EXPR va_list
                va_list CursorKind.TYPE_REF va_list
                 CursorKind.INTEGER_LITERAL int
       CursorKind.RETURN_STMT 
        _Result CursorKind.UNEXPOSED_EXPR int
          _Result CursorKind.DECL_REF_EXPR int
  _printf_s_l CursorKind.FUNCTION_DECL int (const char *const, const _locale_t, ...)
    _Format CursorKind.PARM_DECL const char *const
    _Locale CursorKind.PARM_DECL const _locale_t
      _locale_t CursorKind.TYPE_REF _locale_t
     CursorKind.COMPOUND_STMT 
       CursorKind.DECL_STMT 
        _Result CursorKind.VAR_DECL int
       CursorKind.DECL_STMT 
        _ArgList CursorKind.VAR_DECL va_list
          va_list CursorKind.TYPE_REF va_list
       CursorKind.PAREN_EXPR void
         CursorKind.CSTYLE_CAST_EXPR void
           CursorKind.PAREN_EXPR void
            __v

In [5]:
def get_function_params(node, function_name, result):
    try:
        if node.kind == CursorKind.FUNCTION_DECL and node.spelling == function_name:
            for c in node.get_children():
                if c.kind == CursorKind.PARM_DECL:
                    # print(c.spelling + " " + str(c.kind) + " " + str(c.type.spelling))
                    result.append({"name": c.spelling, "data_type": c.type.spelling})
            return
    except ValueError:
        pass

    for c in node.get_children():
        get_function_params(c, function_name, result)

In [6]:
param_list = []
get_function_params(root_cursor, "getSum", param_list)
param_list

[]

In [7]:
def get_called_functions(root, node, result):
    try:
        if node.kind == CursorKind.CALL_EXPR:
            # "location": node.extent
            current_function = {"name": node.spelling, "return_type": node.type.spelling, "args": []}
            for c in node.get_arguments():
                current_function["args"].append({"name": list(c.get_tokens())[0].spelling if len(list(c.get_tokens())) > 0 \
                                                 else c.spelling, "data_type": c.type.spelling, "cursor_kind": c.kind})
                # current_function["args"].append({"name": c.spelling, "data_type": c.type.spelling, "cursor_kind": c.kind})
                # print(node.location)
                
            current_param_list = []
            get_function_params(root, node.spelling, current_param_list)
            current_function["params"] = current_param_list

            result.append(current_function)
            return
    except ValueError:
        pass

    for c in node.get_children():
        get_called_functions(root, c, result)

In [8]:
function_list = []
get_called_functions(root_cursor, root_cursor, function_list)
function_list

[{'name': '__stdio_common_vfwprintf',
  'return_type': 'int',
  'args': [{'name': '',
    'data_type': 'unsigned long long',
    'cursor_kind': CursorKind.UNEXPOSED_EXPR},
   {'name': '_Stream',
    'data_type': 'FILE *',
    'cursor_kind': CursorKind.UNEXPOSED_EXPR},
   {'name': '_Format',
    'data_type': 'const wchar_t *',
    'cursor_kind': CursorKind.UNEXPOSED_EXPR},
   {'name': '_Locale',
    'data_type': '_locale_t',
    'cursor_kind': CursorKind.UNEXPOSED_EXPR},
   {'name': '_ArgList',
    'data_type': 'va_list',
    'cursor_kind': CursorKind.UNEXPOSED_EXPR}],
  'params': [{'name': '_Options', 'data_type': 'unsigned long long'},
   {'name': '_Stream', 'data_type': 'FILE *'},
   {'name': '_Format', 'data_type': 'const wchar_t *'},
   {'name': '_Locale', 'data_type': '_locale_t'},
   {'name': '_ArgList', 'data_type': 'va_list'}]},
 {'name': '_vfwprintf_l',
  'return_type': 'int',
  'args': [{'name': '_Stream',
    'data_type': 'FILE *',
    'cursor_kind': CursorKind.UNEXPOSED_EXP

In [9]:
for function in function_list:
        
        if len(function["args"]) == 2 and \
            (function["args"][0]["data_type"] == function["args"][1]["data_type"]) and \
            (function["args"][0]["name"] != function["args"][1]["name"]):
            
            positive_sample = [function["name"], function["args"][0]["name"], function["args"][1]["name"],
                              function["return_type"]]
            
            if(len(function["params"]) == 2):
                positive_sample.append(function["params"][0]["name"])
                positive_sample.append(function["params"][1]["name"])
            else:
                positive_sample.append("")
                positive_sample.append("")
            
            negative_sample = deepcopy(positive_sample)
            # swap
            negative_sample[1], negative_sample[2] = negative_sample[2], negative_sample[1]
            
            positive_sample.append(str(0))
            negative_sample.append(str(1))
            
            positive_sample = ",".join(positive_sample)
            negative_sample = ",".join(negative_sample)
            
            print(positive_sample)
            print(negative_sample)
            

---------
## Negative sample generation from all code snippets - Function args swap only
---------

In [10]:
def generate_function_args_swap_dataset(root_dir):
    total_files, total_samples = 0, 0
    
    with open("function_swap_samples.txt", 'a') as function_swap_samples:
        #function_swap_samples.write("samples\n")
        
        for root, dirs, files in os.walk(root_dir):
                for file in files:
                    if file.endswith(".c"):
                        total_files += 1

                        file_path = os.path.join(root, file)
                        with open(file_path, 'rb') as f:
                            # content = str(f.read())
                            start_cursor = index.parse(f.name).cursor

                            function_list = []
                            get_called_functions(start_cursor, start_cursor, function_list)

                            for function in function_list:

                                    if len(function["args"]) == 2 and \
                                        (function["args"][0]["data_type"] == function["args"][1]["data_type"]) and \
                                        (function["args"][0]["name"] != function["args"][1]["name"]):

                                        positive_sample = [function["name"], function["args"][0]["name"], function["args"][1]["name"],
                                                          function["return_type"]]

                                        if(len(function["params"]) == 2):
                                            positive_sample.append(function["params"][0]["name"])
                                            positive_sample.append(function["params"][1]["name"])
                                        else:
                                            positive_sample.append("")
                                            positive_sample.append("")

                                        negative_sample = deepcopy(positive_sample)
                                        # swap
                                        negative_sample[1], negative_sample[2] = negative_sample[2], negative_sample[1]

                                        positive_sample.append(str(0))
                                        negative_sample.append(str(1))

                                        positive_sample = ",".join(positive_sample)
                                        negative_sample = ",".join(negative_sample)
                                        
                                        function_swap_samples.write(positive_sample + "\n")
                                        function_swap_samples.write(negative_sample + "\n")

                    if total_files % 100 == 0:
                        print("Total files:", total_files, ",", "Total samples:", total_samples)

In [11]:
root_dir = "D:\\cleaned\\"
generate_function_args_swap_dataset(root_dir)